In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
sys.path.append('../experiments')
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
from neuro.features import qa_questions, feature_spaces
from neuro.data import story_names
from neuro.features.stim_utils import load_story_wordseqs, load_story_wordseqs_huge
import random
import json
import joblib
from tqdm import tqdm
from collections import defaultdict
fit_encoding = __import__('02_fit_encoding')

### Get data

In [ ]:
# story_names_list = story_names.get_story_names(
# 'shared', 'train', use_huge=True)
story_names_list = story_names.get_story_names(
    all=True)
print('loaded', len(story_names_list), 'stories')
# wordseqs = feature_spaces.get_story_wordseqs(story_names_train)
wordseqs = load_story_wordseqs_huge(story_names_list)

ngrams_list_total = []
for story in story_names_list:
    ngrams_list = feature_spaces.get_ngrams_list_main(
        wordseqs[story], num_ngrams_context=10)
    # print(ngrams_list[:10])
    ngrams_list_total.extend(ngrams_list)

questions = qa_questions.get_questions('v3_boostexamples', full=True)

prompt_template = 'Read the input then answer a question about the input.\nInput: {example}\nQuestion: {question} Answer yes or no.'
prompt = prompt_template.format(
    example=ngrams_list[100], question='Does the sentence include a metaphor?')
# print(prompt_template.format(example=ngrams_list[100], question=questions[0]))
print(prompt)

In [ ]:
rng = np.random.RandomState(42)
# select 100 random examples
ngrams_list_sample = rng.choice(ngrams_list_total, 100, replace=False)

In [ ]:
ngrams_list_sample

# Estimate cost for running gpt-4 on all of these...

In [ ]:
lm = imodelsx.llm.get_llm('gpt-4')

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

In [ ]:
resp = lm(messages, max_new_tokens=1, temperature=0, return_str=False)

In [ ]:
input_tokens = len(ngrams_list_total) * 56
print(len(ngrams_list_total), 'inputs')
print(f'{input_tokens:,} tokens')
print(f'{100 * input_tokens:,} tokens')